# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/11_series_temporelles/11_demo_arima_prophet.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '11_demo_arima_prophet.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 12 - Séries Temporelles : ARIMA & Prophet

**Objectifs :**
- Analyser et visualiser des séries temporelles
- Tester la stationnarité (ADF test)
- Décomposer en tendance, saisonnalité, résidu
- Modéliser avec ARIMA et SARIMA
- Utiliser Prophet de Facebook pour forecasting
- Évaluer les prédictions avec métriques appropriées

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Séries temporelles
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Prophet
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    print("Prophet non installé. Pour l'installer: pip install prophet")
    PROPHET_AVAILABLE = False

# Métriques
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Bibliothèques chargées avec succès")

## 1. Génération de Données Synthétiques

Créons une série temporelle avec tendance, saisonnalité et bruit.

In [ ]:
def generate_timeseries(n=365*3, trend_coef=0.1, seasonal_period=365, noise_std=10, seed=42):
    """
    Génère une série temporelle avec tendance, saisonnalité et bruit
    
    Paramètres:
    - n: nombre de points
    - trend_coef: coefficient de tendance linéaire
    - seasonal_period: période de la saisonnalité
    - noise_std: écart-type du bruit
    """
    np.random.seed(seed)
    
    # Dates
    dates = pd.date_range(start='2021-01-01', periods=n, freq='D')
    
    # Tendance linéaire
    t = np.arange(n)
    trend = trend_coef * t
    
    # Saisonnalité (annuelle)
    seasonality = 20 * np.sin(2 * np.pi * t / seasonal_period)
    
    # Bruit
    noise = np.random.normal(0, noise_std, n)
    
    # Série complète
    y = 100 + trend + seasonality + noise
    
    # DataFrame
    df = pd.DataFrame({
        'date': dates,
        'value': y,
        'trend': 100 + trend,
        'seasonality': seasonality,
        'noise': noise
    })
    df.set_index('date', inplace=True)
    
    return df

# Générer données
df = generate_timeseries(n=365*3, trend_coef=0.05, seasonal_period=365, noise_std=8)

print(f"Série temporelle générée: {len(df)} observations")
print(f"Période: {df.index.min()} à {df.index.max()}")
print(f"\nPremières observations:")
print(df.head())

In [ ]:
# Visualisation
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Série complète
axes[0].plot(df.index, df['value'], label='Série temporelle', color='blue')
axes[0].set_title('Série Temporelle Complète', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Valeur')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Tendance
axes[1].plot(df.index, df['trend'], label='Tendance', color='green')
axes[1].set_title('Composante Tendance', fontsize=12)
axes[1].set_ylabel('Tendance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Saisonnalité
axes[2].plot(df.index, df['seasonality'], label='Saisonnalité', color='orange')
axes[2].set_title('Composante Saisonnalité', fontsize=12)
axes[2].set_ylabel('Saisonnalité')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Bruit
axes[3].plot(df.index, df['noise'], label='Bruit', color='red', alpha=0.5)
axes[3].set_title('Composante Bruit', fontsize=12)
axes[3].set_ylabel('Bruit')
axes[3].set_xlabel('Date')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Analyse Exploratoire

### 2.1 Statistiques Descriptives

In [ ]:
print("=== Statistiques Descriptives ===")
print(df['value'].describe())
print(f"\nSkewness: {df['value'].skew():.4f}")
print(f"Kurtosis: {df['value'].kurtosis():.4f}")

### 2.2 Décomposition de la Série

In [ ]:
# Décomposition additive
decomposition = seasonal_decompose(df['value'], model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(14, 10))

decomposition.observed.plot(ax=axes[0], title='Série Observée', color='blue')
axes[0].set_ylabel('Observée')
axes[0].grid(True, alpha=0.3)

decomposition.trend.plot(ax=axes[1], title='Tendance', color='green')
axes[1].set_ylabel('Tendance')
axes[1].grid(True, alpha=0.3)

decomposition.seasonal.plot(ax=axes[2], title='Saisonnalité', color='orange')
axes[2].set_ylabel('Saisonnalité')
axes[2].grid(True, alpha=0.3)

decomposition.resid.plot(ax=axes[3], title='Résidus', color='red')
axes[3].set_ylabel('Résidus')
axes[3].set_xlabel('Date')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.3 Test de Stationnarité (ADF)

In [ ]:
def adf_test(timeseries, name=''):
    """
    Test de Dickey-Fuller Augmenté pour la stationnarité
    """
    print(f"\n=== Test ADF {name} ===")
    result = adfuller(timeseries.dropna())
    
    print(f"ADF Statistic: {result[0]:.6f}")
    print(f"p-value: {result[1]:.6f}")
    print(f"Lags utilisés: {result[2]}")
    print(f"Nombre d'observations: {result[3]}")
    
    print("\nValeurs critiques:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.3f}")
    
    if result[1] < 0.05:
        print("\n✅ Résultat: Série STATIONNAIRE (rejeter H0, p < 0.05)")
    else:
        print("\n❌ Résultat: Série NON-STATIONNAIRE (ne pas rejeter H0, p >= 0.05)")
    
    return result

# Test sur série originale
adf_result = adf_test(df['value'], name='Série Originale')

### 2.4 Stationnarisation par Différenciation

In [ ]:
# Première différenciation
df['value_diff1'] = df['value'].diff()

# Deuxième différenciation (si nécessaire)
df['value_diff2'] = df['value_diff1'].diff()

# Visualisation
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

axes[0].plot(df.index, df['value'], color='blue')
axes[0].set_title('Série Originale', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Valeur')
axes[0].grid(True, alpha=0.3)

axes[1].plot(df.index, df['value_diff1'], color='green')
axes[1].set_title('Après 1ère Différenciation', fontsize=12)
axes[1].set_ylabel('Différence 1')
axes[1].grid(True, alpha=0.3)

axes[2].plot(df.index, df['value_diff2'], color='orange')
axes[2].set_title('Après 2ème Différenciation', fontsize=12)
axes[2].set_ylabel('Différence 2')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tests ADF
adf_test(df['value_diff1'], name='Après 1ère Différenciation')
adf_test(df['value_diff2'], name='Après 2ème Différenciation')

### 2.5 ACF et PACF

In [ ]:
# ACF et PACF de la série différenciée
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ACF
plot_acf(df['value_diff1'].dropna(), lags=40, ax=axes[0])
axes[0].set_title('Autocorrélation (ACF)', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# PACF
plot_pacf(df['value_diff1'].dropna(), lags=40, ax=axes[1])
axes[1].set_title('Autocorrélation Partielle (PACF)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Modèle ARIMA

### 3.1 Sélection des Paramètres (p, d, q)

In [ ]:
# Train/Test Split (80/20)
train_size = int(len(df) * 0.8)
train = df['value'][:train_size]
test = df['value'][train_size:]

print(f"Train size: {len(train)} observations")
print(f"Test size: {len(test)} observations")
print(f"Train period: {train.index.min()} à {train.index.max()}")
print(f"Test period: {test.index.min()} à {test.index.max()}")

In [ ]:
# Grid search pour trouver les meilleurs paramètres ARIMA
def arima_grid_search(train_data, p_range, d_range, q_range):
    """
    Grid search pour sélectionner les meilleurs paramètres ARIMA
    """
    best_aic = np.inf
    best_params = None
    best_model = None
    
    results = []
    
    for p in p_range:
        for d in d_range:
            for q in q_range:
                try:
                    model = ARIMA(train_data, order=(p, d, q))
                    fitted = model.fit()
                    aic = fitted.aic
                    
                    results.append({
                        'p': p, 'd': d, 'q': q,
                        'AIC': aic,
                        'BIC': fitted.bic
                    })
                    
                    if aic < best_aic:
                        best_aic = aic
                        best_params = (p, d, q)
                        best_model = fitted
                        
                except Exception as e:
                    continue
    
    return best_params, best_model, pd.DataFrame(results)

# Recherche
print("Recherche des meilleurs paramètres ARIMA...")
best_params, best_model, results_df = arima_grid_search(
    train,
    p_range=range(0, 4),
    d_range=range(0, 3),
    q_range=range(0, 4)
)

print(f"\nMeilleurs paramètres: ARIMA{best_params}")
print(f"AIC: {best_model.aic:.2f}")
print(f"BIC: {best_model.bic:.2f}")

# Top 10 modèles
print("\nTop 10 modèles (par AIC):")
print(results_df.sort_values('AIC').head(10))

### 3.2 Entraînement et Prédiction

In [ ]:
# Résumé du modèle
print(best_model.summary())

In [ ]:
# Prédictions sur test set
forecast = best_model.forecast(steps=len(test))

# Visualisation
plt.figure(figsize=(14, 6))
plt.plot(train.index, train, label='Train', color='blue')
plt.plot(test.index, test, label='Test (Réel)', color='green')
plt.plot(test.index, forecast, label=f'Prédictions ARIMA{best_params}', color='red', linestyle='--')
plt.axvline(train.index[-1], color='black', linestyle=':', label='Train/Test Split')
plt.title(f'Prédictions ARIMA{best_params}', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Valeur')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.3 Évaluation

In [ ]:
def evaluate_forecast(y_true, y_pred, model_name='Modèle'):
    """
    Calcule et affiche les métriques de forecasting
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n=== Métriques {model_name} ===")
    print(f"MAE (Mean Absolute Error):       {mae:.4f}")
    print(f"MSE (Mean Squared Error):        {mse:.4f}")
    print(f"RMSE (Root Mean Squared Error):  {rmse:.4f}")
    print(f"MAPE (Mean Absolute % Error):    {mape:.2f}%")
    
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape}

arima_metrics = evaluate_forecast(test.values, forecast.values, f'ARIMA{best_params}')

### 3.4 Diagnostic des Résidus

In [ ]:
# Diagnostic plot
best_model.plot_diagnostics(figsize=(14, 10))
plt.tight_layout()
plt.show()

## 4. Modèle SARIMA (avec Saisonnalité)

Pour capturer la saisonnalité annuelle.

In [ ]:
# SARIMA avec saisonnalité annuelle (période = 365 jours)
# Paramètres: SARIMA(p,d,q)(P,D,Q,s)
sarima_order = (1, 1, 1)  # Non-saisonnier
seasonal_order = (1, 1, 1, 365)  # Saisonnier avec période 365

print(f"Entraînement SARIMA{sarima_order}x{seasonal_order}...")

sarima_model = SARIMAX(train, 
                       order=sarima_order,
                       seasonal_order=seasonal_order)
sarima_fitted = sarima_model.fit(disp=False)

print(f"\nAIC: {sarima_fitted.aic:.2f}")
print(f"BIC: {sarima_fitted.bic:.2f}")

In [ ]:
# Prédictions SARIMA
sarima_forecast = sarima_fitted.forecast(steps=len(test))

# Visualisation
plt.figure(figsize=(14, 6))
plt.plot(train.index, train, label='Train', color='blue')
plt.plot(test.index, test, label='Test (Réel)', color='green')
plt.plot(test.index, sarima_forecast, label=f'SARIMA{sarima_order}x{seasonal_order}', 
         color='purple', linestyle='--')
plt.axvline(train.index[-1], color='black', linestyle=':', label='Train/Test Split')
plt.title('Prédictions SARIMA', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Valeur')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Métriques
sarima_metrics = evaluate_forecast(test.values, sarima_forecast.values, 'SARIMA')

## 5. Prophet de Facebook

Framework simple et robuste pour le forecasting avec saisonnalités multiples.

In [ ]:
if PROPHET_AVAILABLE:
    # Préparer les données au format Prophet (colonnes 'ds' et 'y')
    train_prophet = pd.DataFrame({
        'ds': train.index,
        'y': train.values
    })
    
    # Créer et entraîner le modèle
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='additive',
        changepoint_prior_scale=0.05  # Flexibilité des changepoints
    )
    
    print("Entraînement du modèle Prophet...")
    prophet_model.fit(train_prophet)
    print("✅ Entraînement terminé")
else:
    print("⚠️ Prophet non disponible. Installer avec: pip install prophet")

In [ ]:
if PROPHET_AVAILABLE:
    # Créer dataframe pour prédictions futures
    future = prophet_model.make_future_dataframe(periods=len(test), freq='D')
    
    # Prédictions
    prophet_forecast = prophet_model.predict(future)
    
    # Extraire prédictions sur test set
    prophet_test_pred = prophet_forecast.iloc[-len(test):]['yhat'].values
    
    # Visualisation Prophet
    fig1 = prophet_model.plot(prophet_forecast, figsize=(14, 6))
    plt.title('Prédictions Prophet', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Composantes (tendance + saisonnalité)
    fig2 = prophet_model.plot_components(prophet_forecast, figsize=(14, 8))
    plt.tight_layout()
    plt.show()
    
    # Métriques
    prophet_metrics = evaluate_forecast(test.values, prophet_test_pred, 'Prophet')

## 6. Comparaison des Modèles

In [ ]:
# Tableau comparatif
comparison = pd.DataFrame({
    f'ARIMA{best_params}': arima_metrics,
    'SARIMA': sarima_metrics,
})

if PROPHET_AVAILABLE:
    comparison['Prophet'] = prophet_metrics

print("\n=== Comparaison des Modèles ===")
print(comparison.T)

# Meilleur modèle (RMSE)
best_model_name = comparison.loc['RMSE'].idxmin()
print(f"\n🏆 Meilleur modèle (RMSE): {best_model_name}")

In [ ]:
# Visualisation comparative
plt.figure(figsize=(14, 7))
plt.plot(train.index, train, label='Train', color='blue', alpha=0.7)
plt.plot(test.index, test, label='Test (Réel)', color='green', linewidth=2)
plt.plot(test.index, forecast, label=f'ARIMA{best_params}', 
         color='red', linestyle='--', alpha=0.8)
plt.plot(test.index, sarima_forecast, label='SARIMA', 
         color='purple', linestyle='--', alpha=0.8)

if PROPHET_AVAILABLE:
    plt.plot(test.index, prophet_test_pred, label='Prophet', 
             color='orange', linestyle='--', alpha=0.8)

plt.axvline(train.index[-1], color='black', linestyle=':', linewidth=2, label='Train/Test Split')
plt.title('Comparaison des Prédictions - ARIMA vs SARIMA vs Prophet', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Valeur')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Forecasting Multi-Horizons

In [ ]:
# Prédire sur plusieurs horizons (7, 30, 90 jours)
horizons = [7, 30, 90, 180]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, horizon in enumerate(horizons):
    # ARIMA forecast
    forecast_h = best_model.forecast(steps=horizon)
    
    # Plot
    ax = axes[idx]
    ax.plot(train.index[-180:], train.values[-180:], label='Train (derniers 180 jours)', color='blue')
    
    # Dates futures
    future_dates = pd.date_range(start=train.index[-1] + timedelta(days=1), periods=horizon, freq='D')
    ax.plot(future_dates, forecast_h, label=f'Prédiction {horizon}j', color='red', linestyle='--', marker='o')
    
    ax.axvline(train.index[-1], color='black', linestyle=':', label='Début Forecast')
    ax.set_title(f'Horizon: {horizon} jours', fontsize=12, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Valeur')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

Dans ce notebook, nous avons exploré :

1. **Analyse exploratoire** : décomposition, stationnarité, ACF/PACF
2. **ARIMA** : modèle classique pour séries stationnaires
3. **SARIMA** : extension avec saisonnalité
4. **Prophet** : framework robuste de Facebook pour forecasting
5. **Évaluation** : MAE, RMSE, MAPE
6. **Comparaison** : analyse des performances de chaque modèle

**Points clés :**
- Toujours tester la stationnarité avant ARIMA
- Utiliser ACF/PACF pour guider la sélection de (p, q)
- SARIMA pour saisonnalités claires
- Prophet excellent pour données réelles avec outliers et manquantes
- Valider avec Time Series Split (pas de CV classique)

**Prochaine étape** : Deep Learning pour séries temporelles (LSTM, GRU)